###Setup
1. Importing Libraries
2. Mounting Drive
3. Defining `BASE_PATH` & `TARGET_COUNTRY`

In [ ]:
# IF RUNNING THE NEW MODEL
!pip install flair==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=b256f82663f5f8bab2e2781253fa2cb589065fc4bdcdbcce65a9b1002f10712a
  Stored in directory: /root/.cache/pip

In [ ]:
# IF RUNNING THE OLD MODEL
#!pip install flair==0.12.2

In [ ]:
import pandas as pd
import requests
import json
import os
import tqdm
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
import matplotlib.pyplot as plt
from typing import List
from nltk.tokenize import word_tokenize
from random import choice
import re
from pathlib import Path
import joblib
import json
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ValueError: mount failed

In [ ]:
TARGET_COUNTRY = 'be'
BASE_PATH = Path('/content/drive/MyDrive/deep-learning/transactions/')
DATASET_PATH = BASE_PATH / 'Datasets-Versions'

###DF Parsing

####Building cleaned_text_bert

In [ ]:
df = pd.read_csv(BASE_PATH/'datasets'/'df_final.csv')

In [ ]:
#filter to country
df = df[df['country'] == TARGET_COUNTRY]

In [ ]:
#organize df
df = df[['country',	'language',	'VATType',	'baseCurrencyAmount',	'category_id',	'APC1',	'APC2',	'counterparty_name',	'communication']]

In [ ]:
iban_re = re.compile(
    r'\b'                                            # Anchor: Start at a word boundary
    r'(?:'                                           # Start non-capturing group for the two formats
      r'[A-Z]{2}[0-9]{2}'                            # Mandatory start: Country Code (AA) + Check Digits (##)
      r'(?:[\s]?[A-Z0-9]{4}){2,7}'                   # 2 to 7 groups of (optional space + 4 alphanumeric chars)
      r'(?:[\s]?[A-Z0-9]{1,4})?'                     # Optional final group of 1 to 4 chars, optionally preceded by a space
    r'|'                                             # OR
      r'[A-Z]{2}[0-9]{2}[A-Z0-9]{10,30}'             # AA## followed by 10 to 30 contiguous alphanumeric chars
    r')'
    r'\b',                                           # Anchor: End at a word boundary
    flags=re.IGNORECASE                              # Match case-insensitively (most IBAN parsers ignore case on letters)
)

In [ ]:
big_num_re = re.compile(r'\d{5,}')

In [ ]:
dates_re = re.compile(
    r'\d{4}-\d{2}-\d{2}'   # yyyy-mm-dd
    r'|'
    r'\d{4}/\d{2}/\d{2}'   # yyyy/mm/dd
    r'|'
    r'\d{4}\.\d{2}\.\d{2}' # yyyy.mm.dd
    r'|'
    r'\d{2}-\d{2}-\d{4}'   # dd-mm-yyyy
    r'|'
    r'\d{2}/\d{2}/\d{4}'   # dd/mm/yyyy
    r'|'
    r'\d{2}\.\d{2}\.\d{4}' # dd.mm.yyyy
    r'|'
    r'\b\d{2}[-/\.]\d{2}[-/\.]\d{2}\b'      # 02-10-23, 10/02/23, etc.
    r'|'
    r'\b\d{1,2}\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*[\s,]+\d{4}\b'   # 02 Oct 2023, 2 October 2023
    r'|'
    r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\s+\d{1,2},?\s+\d{4}\b'    # Oct 02, 2023, October 2 2023
    r'|'
    r'\b\d{4}-\d{2}-\d{2}[T\s]\d{2}:\d{2}(:\d{2})?(\.\d+)?(Z|[+-]\d{2}:?\d{2})?\b'
)


In [ ]:
url_re = re.compile(
    r'(?:https?://|ftp://|ftps://|sftp://|file://|mailto:|tel:|sms:)?(?:www\.|m\.)?',
    re.IGNORECASE
)

In [ ]:
fin_re = re.compile(
    r'\b(?:'
    r'sumup'
    r'|pluxee'
    r'|paypal(?:\s+europe)?(?:\s+s\.?a\.?r\.?l\.?)?(?:\s+et\s+cie)?(?:\s+s\.?c\.?a\.?)?'  # PayPal legal variants
    r'|ing(?:\s+app)?'
    r'|kbc(?:\s+mobile|\s+brussels|\s+betaalknop)?'
    r'|crelan'
    r'|kredbebb\w*'
    r'|bancontact'
    r'|maestro'
    r'|debit\s+mastercard'
    r'|mastercard'
    r'|visa'
    r'|apple\s*pay'
    r'|\bpaypal\b[\s,;\-]*'
    r'|paddlenet\s*[\n,;\-]*\s*(.+)'
    r'|google\s*g\s*suite[\s_+\-]*([\w\s]+)'
    r')\b',
    flags=re.IGNORECASE
)

In [ ]:
punct_re = re.compile(r'[^\w\s\&]')

In [ ]:
def preprocess_text(communication, counterparty_name) -> str:
    """
    Combines communication + counterparty_name, fixes encoding,
    removes big numbers/dates, cleans text for BERT.
    """

    comm = '' if pd.isna(communication) else str(communication)
    cp   = '' if pd.isna(counterparty_name) else str(counterparty_name)

    combined = (cp + ' ' + comm).strip()
    combined = fix_encoding(combined)
    combined = combined.lower()
    combined = fin_re.sub('', combined)
    combined = iban_re.sub('', combined)
    combined = big_num_re.sub('', combined)
    combined = dates_re.sub('', combined)
    combined = url_re.sub('', combined)
    combined = punct_re.sub(' ', combined)
    combined = re.sub(r'\s+', ' ', combined).strip()

    return combined

In [ ]:
df['cleaned_text_bert'] = df.apply(lambda x: preprocess_text(x['communication'], x['counterparty_name']), axis = 1)

####Truncate long cleaned_text_bert

In [ ]:
#truncate to max 200 length inputs
df = df[df['cleaned_text_bert'].str.len() < 200]

In [ ]:
#drop unkown and Nan in cleaned_text_bert
df = df[~(df['counterparty_name'].isna() & df['communication'].isna())]
df = df[~(df['cleaned_text_bert'] == '')]

In [ ]:
#checkpoint
df.to_csv(DATASET_PATH/f'df_{TARGET_COUNTRY}_parsed_no_labels.csv')

####Run Old model for Baseline

In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence

In [ ]:
#loading model
model_ner = SequenceTagger.load(BASE_PATH/'old-models'/f'flair_ner_{TARGET_COUNTRY}.pt')

In [ ]:
#running predictions
sentences = [Sentence(str(text)) for text in df['cleaned_text_bert'].to_list()]
model_ner.predict(sentences, mini_batch_size=4096, verbose=False)

In [ ]:
#adding predictions to df
df['old_model_results'] = [sentence.get_labels('ner')[0].data_point.text if len(sentence.get_labels('ner')) > 0 else '' for sentence in sentences]

In [ ]:
#checkpoint
df.to_csv(BASE_PATH/'Datasets-Versions'/f'df_{TARGET_COUNTRY}_parsed_old_labels.csv')

We then generated the labels for the NER DataFrame with GPT:
- `df_be_gpt4o_mini_ner.csv` & `df_be_gpt4o_mini_ner2.csv` (For **BELGIUM**)
- `df_de_gpt4o_mini_ner.csv` (For **GERMANY**)

Belgium has 2 files as the DataFrame was properly massive

####Jaro-Winkler approach

In [ ]:
df1 = pd.read_csv(BASE_PATH/f'Datasets-Versions/df_{TARGET_COUNTRY}_gpt4o_mini_ner.csv')
df2 = pd.read_csv(BASE_PATH/f'Datasets-Versions/df_{TARGET_COUNTRY}_gpt4o_mini_ner2.csv')

In [ ]:
df = pd.concat([df1, df2])

In [ ]:
df = df[df['extracted_organization'].notna()]

In [ ]:
df.drop_duplicates(subset=['hash', 'cleaned_text_bert', 'APC1', 'APC2', 'VATType',	'baseCurrencyAmount',	'category_id'], keep='first', inplace=True)

In [ ]:
!pip install jarowinkler
from jarowinkler import *

In [ ]:
df.shape

(741958, 14)

In [ ]:
df.dropna(subset=['old_model_results'], inplace=True)

In [ ]:
def old_model_pred(text):
    matches = re.findall(r'Span\[\d+:\d+\]:\s*["\']([^"\']+)["\']', text)
    return matches[0] if matches else None

df['old_model_results'] = df['old_model_results'].apply(old_model_pred)

In [ ]:
df.rename(columns = {'extracted_organization':'gpt_generated_labels'}, inplace = True)

In [ ]:
df['gpt_generated_labels'] = df['gpt_generated_labels'].astype(str).apply(lambda x: x.lower())

In [ ]:
#compute jaro similarity between gpt_labels & old_model_predictions
df['jaro_score'] = df.apply(lambda row: jarowinkler_similarity(row['old_model_results'], row['gpt_generated_labels']), axis = 1)

In [ ]:
#filter df to similarity between 0.8-1
df = df[df['jaro_score']>=0.8]

In [ ]:
df.shape

(401994, 15)

####Pos Tagging

In [ ]:
def pos_tagging(row):
    text = str(row['cleaned_text_bert']).lower().split()
    label = str(row['gpt_generated_labels']).strip().lower()

    if not label:
        return ' '.join(['O'] * len(text))

    label_words = label.split()
    pos = ['O'] * len(text)
    label_len = len(label_words)

    for i in range(len(text) - label_len + 1):
        if text[i:i + label_len] == label_words:
            pos[i] = 'B-ORG'
            for j in range(1, label_len):
                pos[i + j] = 'I-ORG'

    return ','.join(pos)


In [ ]:
from tqdm import tqdm
tqdm.pandas()
df['TAG'] = df.progress_apply(pos_tagging, axis=1)

In [ ]:
df = df[~ (df['gpt_generated_labels'] == "unknown")]

In [ ]:
def ner_tags_id(x):
  words_tags = []
  words = x.split(',')
  for w in words:
    if w == 'O':
      words_tags.append(0)
    if w == 'B-ORG':
      words_tags.append(1)
    if w == 'I-ORG':
      words_tags.append(2)
  return words_tags

In [ ]:
df['ner_tags'] = df['TAG'].apply(ner_tags_id)

In [ ]:
df["tokens"] = df["cleaned_text_bert"].astype(str).apply(lambda x: x.split())

In [ ]:
df.to_csv(f"/content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/{TARGET_COUNTRY}_parsed_ner.csv")

In [ ]:
!pip install flair[word-embeddings]

In [ ]:
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger

In [ ]:
df = pd.read_csv(DATASET_PATH / f'{TARGET_COUNTRY}_parsed_ner.csv')

In [ ]:
df = df[['cleaned_text_bert', 'gpt_generated_labels']].copy()

In [ ]:
# Filter to labels in cleaned_text_bert for each row
df = df[df.apply(lambda row: str(row['cleaned_text_bert']).lower().find(str(row['gpt_generated_labels']).lower()) != -1, axis=1)]

In [ ]:
df.shape

(377259, 2)

In [ ]:
def format_annotation(x):
    try:
        return [(str(x).lower().strip(), 'ORG')]
    except:
        print(x)
        raise Exception('error')

In [ ]:
df['gpt_generated_labels'] = df.gpt_generated_labels.apply(format_annotation)
df.columns = ['text', 'annotation']

In [ ]:
df['text'] = df.text.apply(lambda x: str(x).lower())

In [ ]:
df.loc[df.text.isnull()]
df = df.loc[df.text != '']

In [ ]:
from difflib import SequenceMatcher

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)

    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme.
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'

    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

In [ ]:
def create_data(df, filepath):
  '''
  The function responsible for the creation of data in the flair required format.
  '''
  with open(filepath , 'w') as f:
    for text, annotation in zip(df.text, df.annotation):
      text_ = text
      match_list = []
      for i in annotation:
        a, text_ = matcher(text, i[0])
        try:
          match_list.append((a[0][0], a[0][1], i[1]))
        except Exception as e:
          print(a)
          print(text_)
          raise Exception(e)

      d = mark_sentence(text, match_list)

      for i in d.keys():
        f.writelines(i + ' ' + d[i] +'\n')
      f.writelines('\n')

In [ ]:
!cd /content/flair_ner_be && ls -la

/bin/bash: line 1: cd: /content/flair_ner_be: No such file or directory


In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42, shuffle=True)

In [ ]:
data_folder = Path(f'/content/flair_ner_{TARGET_COUNTRY}')
data_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
create_data(train_df, data_folder / 'train.txt')
create_data(test_df, data_folder / 'test.txt')
create_data(val_df, data_folder / 'dev.txt')

###Training

In [ ]:
from flair.datasets import ColumnCorpus
data_folder = Path('/content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be')  # Modifica con il tuo percorso

columns = {0: 'text', 1: 'ner'}
corpus = ColumnCorpus(
    data_folder,
    columns,
    train_file='train.txt',
    test_file='test.txt',
    dev_file='dev.txt'
)
tag_dictionary = corpus.make_label_dictionary(label_type='ner')

2025-11-17 00:56:45,076 Reading data from /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be
2025-11-17 00:56:45,077 Train: /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be/train.txt
2025-11-17 00:56:45,077 Dev: /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be/dev.txt
2025-11-17 00:56:45,077 Test: /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be/test.txt
2025-11-17 00:57:24,981 Computing label dictionary. Progress:


0it [00:00, ?it/s]
264081it [00:05, 46010.21it/s]

2025-11-17 00:57:30,734 Dictionary created for label 'ner' with 1 values: ORG (seen 266425 times)


In [ ]:
embedding_types = [WordEmbeddings('glove'),]
embeddings : StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
tagger : SequenceTagger = SequenceTagger(hidden_size=256, embeddings=embeddings,tag_dictionary=tag_dictionary,tag_type='ner',use_crf=True)

2025-11-17 00:58:22,753 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpguiiluka


100%|██████████| 153M/153M [00:10<00:00, 15.4MB/s]

2025-11-17 00:58:33,657 copying /tmp/tmpguiiluka to cache at /root/.flair/embeddings/glove.gensim.vectors.npy
2025-11-17 00:58:33,736 removing temp file /tmp/tmpguiiluka


2025-11-17 00:58:34,292 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpcc013x2g


100%|██████████| 20.5M/20.5M [00:02<00:00, 9.59MB/s]

2025-11-17 00:58:37,066 copying /tmp/tmpcc013x2g to cache at /root/.flair/embeddings/glove.gensim
2025-11-17 00:58:37,078 removing temp file /tmp/tmpcc013x2g


2025-11-17 00:58:40,321 SequenceTagger predicts: Dictionary with 5 tags: O, S-ORG, B-ORG, E-ORG, I-ORG


In [ ]:
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train(f'flair_ner_{TARGET_COUNTRY}', learning_rate=0.1, mini_batch_size=32, max_epochs=50)

In [ ]:
import shutil

local_model_path = f'flair_ner_{TARGET_COUNTRY}/best-model.pt'
drive_save_path = f'{BASE_PATH}/models/flair_ner_{TARGET_COUNTRY}_best_model.pt'

# Copy the model to Drive
shutil.copy(local_model_path, drive_save_path)

print(f"Model saved to: {drive_save_path}")

Model saved to: /content/drive/MyDrive/deep-learning/transactions/models/flair_ner_be_best_model.pt


###Inference

In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
from flair.datasets import ColumnCorpus

In [ ]:
#Running new model
model_path = f'{BASE_PATH}/models/flair_ner_be_best_model.pt'
model = SequenceTagger.load(model_path)

2025-11-17 10:04:54,997 SequenceTagger predicts: Dictionary with 7 tags: O, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


In [ ]:
data_folder = Path(BASE_PATH/'Datasets-Versions/flair_ner_be')

columns = {0: 'text', 1: 'ner'}
corpus = ColumnCorpus(
    data_folder,
    columns,
    train_file='train.txt',
    test_file='test.txt',
    dev_file='dev.txt'
)

2025-11-17 10:54:29,668 Reading data from /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be
2025-11-17 10:54:29,669 Train: /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be/train.txt
2025-11-17 10:54:29,669 Dev: /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be/dev.txt
2025-11-17 10:54:29,670 Test: /content/drive/MyDrive/deep-learning/transactions/Datasets-Versions/flair_ner_be/test.txt


In [ ]:
from flair.trainers import ModelTrainer

result = model.evaluate(corpus.dev, gold_label_type='ner')
print(f"\n{result.detailed_results}")

100%|██████████| 1769/1769 [01:22<00:00, 21.47it/s]




Results:
- F-score (micro) 0.94
- F-score (macro) 0.94
- Accuracy 0.8868

By class:
              precision    recall  f1-score   support

         ORG     0.9407    0.9394    0.9400     57078

   micro avg     0.9407    0.9394    0.9400     57078
   macro avg     0.9407    0.9394    0.9400     57078
weighted avg     0.9407    0.9394    0.9400     57078



In [ ]:
import torch
from flair.models import SequenceTagger
original_load = torch.load

def patched_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return original_load(*args, **kwargs)
torch.load = patched_load


model_ner = SequenceTagger.load(BASE_PATH/'old-models'/f'flair_ner_{TARGET_COUNTRY}.pt')
torch.load = original_load

from flair.trainers import ModelTrainer
result = model_ner.evaluate(corpus.dev, gold_label_type='ner')
print(f"\n{result.detailed_results}")

2025-11-17 10:57:22,227 SequenceTagger predicts: Dictionary with 7 tags: O, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


100%|██████████| 1769/1769 [01:23<00:00, 21.07it/s]


2025-11-17 10:58:46,585 Evaluating as a multi-label problem: False


Results:
- F-score (micro) 0.6498
- F-score (macro) 0.6498
- Accuracy 0.4813

By class:
              precision    recall  f1-score   support

         ORG     0.6632    0.6370    0.6498     57078

   micro avg     0.6632    0.6370    0.6498     57078
   macro avg     0.6632    0.6370    0.6498     57078
weighted avg     0.6632    0.6370    0.6498     57078

